# Equalização

## Um grupo fazendo duas provas parcialmente distintas

Relizaremos a equalização simultânea e a equalização à posteriori.

Para fazer a equalização simultanea, precisamos estimar os parâmetros dos itens e dos alunos simultâneamente, mesmo aqueles que fazem testes distintos. A equalização será garantida pois haverá algumas questões em comum nos dois testes. Nesse caso, a estimação de parâmetros deve ser feita via máxima verossimilhança marginal.

Sob as seguintes suposições:
* Os respondentes são da mesma população;
* Questões não respondidas por um respondente são independentes de seu traço latente e só não foram respondidas porque não foram apresentadas a esse respondente
* Dois modelos de testes, A e B, que foram construídos de um mesmo banco de questões, sendo:
  * Modelo A: Questões de 1 a 10 do banco de questões,
  * Modelo B: Questões de 7 a 16 do banco de questões,

será realizada uma simulação de equalização.

In [21]:
library(mirt)

# Funcao para dar cbind preenchendo com NA
cbind.fill <- function(...){
    nm <- list(...) 
    nm <- lapply(nm, as.matrix)
    n <- max(sapply(nm, nrow)) 
    do.call(cbind, lapply(nm, function (x) 
        rbind(x, matrix(, n-nrow(x), ncol(x)))))
}

Para gerar o padrão de resposta dos alunos, consideramos que os existem 8 itens médios, 4 itens fáceis e 4 itens difíceis. Faremos dessa forma pois os parâmetros estimados não fariam sentido se gerarmos os padrões de resposta aleatoriamente.

Os itens fáceis são os de número 3, 7, 9 e 16.

Os itens difíceis são os de número 4, 8, 10 e 11.

Os itens médios serão o restante: 1, 2, 5, 6, 12, 13, 14 e 15.

A definição de itens fáceis ou difíceis dependem da estimação dos parâmetros do modelo, que dependerá do padrão de resposta de todos os alunos simultaneamente. Por isso, não necessariamente o que definimos como fácil terá dificuldade baixa, ou vice-versa para o que definimos como difícil. Essa definição foi feita somente para facilitar a criação do padrão de resposta de cada aluno individualmente.

In [22]:
# Dataframe com as respostas dos alunos
data = data.frame('Item 1' = c(1,0,1,NA,NA,NA),
                  'Item 2' = c(0,1,1,NA,NA,NA),
                  'Item 3' = c(1,0,1,NA,NA,NA),
                  'Item 4' = c(1,0,0,NA,NA,NA),
                  'Item 5' = c(1,1,0,NA,NA,NA),
                  'Item 6' = c(0,0,1,NA,NA,NA),
                  'Item 7' = c(1,1,0,1,0,0),
                  'Item 8' = c(1,0,0,1,0,1),
                  'Item 9' = c(0,1,1,1,1,0),
                  'Item 10' = c(0,1,0,1,0,0),
                  'Item 11' = c(NA,NA,NA,1,1,0),
                  'Item 12' = c(NA,NA,NA,0,0,1),
                  'Item 13' = c(NA,NA,NA,1,0,0),
                  'Item 14' = c(NA,NA,NA,1,1,0),
                  'Item 15' = c(NA,NA,NA,0,0,1),
                  'Item 16' = c(NA,NA,NA,1,0,1))
data

Item.1,Item.2,Item.3,Item.4,Item.5,Item.6,Item.7,Item.8,Item.9,Item.10,Item.11,Item.12,Item.13,Item.14,Item.15,Item.16
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,1,1,1,0,1,1,0,0,NA,NA,NA,NA,NA,NA
0,1,0,0,1,0,1,0,1,1,NA,NA,NA,NA,NA,NA
1,1,1,0,0,1,0,0,1,0,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,1,1,1,1,1,0,1,1,0,1
NA,NA,NA,NA,NA,NA,0,0,1,0,1,0,0,1,0,0
NA,NA,NA,NA,NA,NA,0,1,0,0,0,1,0,0,1,1


## Fazendo equalização via população

Primeiro, faremos a estimação dos parâmetros pelo método da máxima verossimilhança marginal (MVM) e utilizar esses valores para estimar os traços latentes.

In [23]:
model = mirt(data, 1, itemtype = '3PL', technical = list(NCYCLES = 1000))

Iteration: 22, Log-Lik: -20.783, Max-Change: 0.000074


Mostrando os parâmetros dos itens.

In [24]:
xi = as.data.frame(coef(model, simplify = TRUE)$items)
xi

,a1,d,g,u
,<dbl>,<dbl>,<dbl>,<dbl>
Item.1,75.3305300,-23.0248414,5.000339e-01,1
Item.2,-78.4216820,23.7288509,2.805644e-06,1
Item.3,75.3305300,-23.0248414,5.000339e-01,1
Item.4,78.4512747,-23.7167923,1.406881e-06,1
Item.5,74.6217391,22.7994007,3.167188e-06,1
Item.6,-75.3454338,-22.8942083,1.553095e-06,1
Item.7,76.5489295,22.7004555,9.305473e-07,1
Item.8,77.4199746,-23.3503427,2.629572e-01,1
Item.9,0.2918373,0.7818701,1.010110e-04,1


O pacote mirt tem uma parametrização diferente para a dificuldade. Na tabela, temos o parâmetro $d$, que é dado por

\begin{align*}
    d = - a\times b \implies b = -\dfrac{d}{a}
\end{align*}

Dito isso, vamos adicionar uma coluna no dataframe $\boldsymbol{\xi}$ de parâmetros dos itens referente a dificuldade e mostrá-lo.

In [25]:
# Na parametrizacao do mirt o parâmetro de dificuldade é dado por d = -a * b e portanto b = - d / a
xi['b'] = - xi['d'] / xi['a1']

Finalmente, estimaremos os traços latentes dos respondentes.

In [26]:
# Agora, estimando os traços latentes
theta = data.frame('respondente' = 1:6)
theta$theta = fscores(model)
theta

respondente,theta
<int>,"<dbl[,1]>"
1,0.73108324
2,0.02872096
3,-1.00461457
4,1.05594111
5,-0.67221722
6,-1.65194901


Os resultados obtidos implicam que o qualto respondente é aquele com maior habilidade, com $\theta_4 \approx 1.056$ e o sexto respondente é aquele com menor habilidade, com $\theta_6 \approx -1.652$, dentre os respondentes de ambos os testes. Ainda, os respondentes do teste A têm, no geral, traço latente maior que os respondentes do teste B. Essa comparação de respondentes dos dois subgrupos é possível devido a equalização dos parâmetros, já que as estimativas dos traços latentes estão na mesma métrica.

Vamos, então, realizar a equalização a posteriori e comparar os resultados.

## Fazendo a equalização a posteriori

Para fazer a equalização a posteriori é preciso que ambos os testes jás estejam calibrados cada um em sua própria escala. Então, a seguir, os testes serão calibrados individualmente via MVM.

A fim de comparar com os resultados da equalização via população, os mesmos padrões de respostas será utilizado.

In [27]:
# Dataframe com as respostas dos respondentes do teste A
data_A = data.frame('Item 1' = c(1,0,1),
                    'Item 2' = c(0,1,1),
                    'Item 3' = c(1,0,1),
                    'Item 4' = c(1,0,0),
                    'Item 5' = c(1,1,0),
                    'Item 6' = c(0,0,1),
                    'Item 7' = c(1,1,0),
                    'Item 8' = c(1,0,0),
                    'Item 9' = c(0,1,1),
                    'Item 10' = c(0,1,0))
data_A

Item.1,Item.2,Item.3,Item.4,Item.5,Item.6,Item.7,Item.8,Item.9,Item.10
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,1,1,1,0,1,1,0,0
0,1,0,0,1,0,1,0,1,1
1,1,1,0,0,1,0,0,1,0


In [28]:
# Dataframe com as respostas dos respondentes do teste B
data_B = data.frame('Item 7' = c(1,0,0),
                    'Item 8' = c(1,0,1),
                    'Item 9' = c(1,1,0),
                    'Item 10' = c(1,0,0),
                    'Item 11' = c(1,1,0),
                    'Item 12' = c(0,0,1),
                    'Item 13' = c(1,0,0),
                    'Item 14' = c(1,1,0),
                    'Item 15' = c(0,0,1),
                    'Item 16' = c(1,0,1))
data_B

Item.7,Item.8,Item.9,Item.10,Item.11,Item.12,Item.13,Item.14,Item.15,Item.16
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1,1,1,0,1,1,0,1
0,0,1,0,1,0,0,1,0,0
0,1,0,0,0,1,0,0,1,1


In [29]:
# Estimacao dos itens via MVM
modelA = mirt(data_A, 1, itemtype = '3PL', technical = list(NCYCLES = 1000))
modelB = mirt(data_B, 1, itemtype = '3PL', technical = list(NCYCLES = 1000))

Iteration: 18, Log-Lik: -10.771, Max-Change: 0.0001049
Iteration: 9, Log-Lik: -6.267, Max-Change: 0.000054


In [30]:
xiA = as.data.frame(coef(modelA, simplify = TRUE)$items)
xiB = as.data.frame(coef(modelB, simplify = TRUE)$items)

xiA['b'] = - xiA['d'] / xiA['a1']
xiB['b'] = - xiB['d'] / xiB['a1']

Agora, fazendo a estimação dos traços latentes.

In [31]:
# Agora, estimando os traços latentes
thetaA = data.frame('respondente' = 1:3)
thetaB = data.frame('respondente' = 4:6)

thetaA$thetaA = fscores(modelA)
thetaB$thetaB = fscores(modelB)

Finalmente, fazendo a equalização utilizando o método média-desvio.

In [32]:
getMD = function(b){
    return(c(mean(b), sd(b)))
}

mdA = getMD(xiA$b)
mdB = getMD(xiB$b)

In [33]:
alpha = mdA[2]/mdB[2]
beta = mdA[1] - alpha * mdB[1]

In [34]:
thetaA_B = data.frame('respondente' = 1:3)
thetaB_A = data.frame('respondente' = 4:6)

thetaA_B$thetaB = alpha*thetaA$thetaA + beta
thetaB_A$thetaA = alpha*thetaB$thetaB + beta

thetaA = rbind(thetaA, thetaB_A)
thetaB = rbind(thetaA_B, thetaB)

## Compração dos resultados

Com o intuito de comparar a equalização em ambos os casos, vamos montar um dataframe com toda as estimativas.

In [35]:
dfthetas = list(thetaA, thetaB)

In [36]:
for (t in dfthetas){
    theta = merge(theta, t, by = 'respondente', all.x = TRUE)
}

Mostrando o dataframe com todas as estimativas, ordenando em ordem crescente os valores dos traços latentes referentes a equalização via população.

In [41]:
round(theta[order(theta$theta),],3)

,respondente,theta,thetaA,thetaB
,<dbl>,"<dbl[,1]>","<dbl[,1]>","<dbl[,1]>"
6,6,-1.652,-0.825,-1.450
3,3,-1.005,-0.618,-0.190
5,5,-0.672,0.078,-0.266
2,2,0.029,-0.618,-0.190
1,1,0.731,1.066,1.094
4,4,1.056,1.044,1.000


Agora, mostrando o dataframe com todas as estimativas, ordenando em ordem crescente os valores dos traços latentes referentes a equalização _a posteriori_ na métrica do grupo A.

In [43]:
round(theta[order(theta$thetaA),],3)

,respondente,theta,thetaA,thetaB
,<dbl>,"<dbl[,1]>","<dbl[,1]>","<dbl[,1]>"
6,6,-1.652,-0.825,-1.450
2,2,0.029,-0.618,-0.190
3,3,-1.005,-0.618,-0.190
5,5,-0.672,0.078,-0.266
4,4,1.056,1.044,1.000
1,1,0.731,1.066,1.094


Finalmente, mostrando o dataframe com todas as estimativas, ordenando em ordem crescente os valores dos traços latentes referentes a equalização _a posteriori_ na métrica do grupo B.

In [44]:
round(theta[order(theta$thetaB),],3)

,respondente,theta,thetaA,thetaB
,<dbl>,"<dbl[,1]>","<dbl[,1]>","<dbl[,1]>"
6,6,-1.652,-0.825,-1.450
5,5,-0.672,0.078,-0.266
2,2,0.029,-0.618,-0.190
3,3,-1.005,-0.618,-0.190
4,4,1.056,1.044,1.000
1,1,0.731,1.066,1.094


No dataframe é possível notar que as equalizações _a posteriori_ produzem resultados consistentes, onde somente o respondente 5 obteve uma nota relativamente menor na métrica do grupo B. Por outro lado, comparando a equalização via população com a equalização _a posteriori_, vemos uma certa consistencia também, mas os resultados são mais divergentes.

É importante enfatizar que em cada caso o pesquisador/avaliador deve olhar os casos divergentes e escolher a equalização que melhor satisfaz suas necessiadades.